In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from collections import Counter

In [2]:
# data
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']


In [3]:
df

,sepal length,sepal width,petal length,petal width,label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [4]:
data = np.array(df.iloc[:100, [0, 1, -1]]) # 取数据集的前100个数据，并取第0,1，-1列
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 自动划分数据集和测试集的方法，测试集比例占0.2

In [5]:
class KNN:
    def __init__(self,x_train,y_train,p=2,k=3): # x为实例点 y为label k为k值 p为距离度量
        self.x=x_train
        self.y=y_train
        self.k=k
        self.p=p
    
    def predict(self,x_pre):
        knn_list=[]
        for j in range(len(self.x)):       
            dis=np.linalg.norm(self.x[j]-x_pre,ord=self.p) # 计算距离，norm函数计算范数，ord=n表示为Ln范数
            knn_list.append((dis,self.y[j]))
       
        sorted_list=sorted(knn_list,key=lambda x:x[0]) # 按照第一列将list进行排序
        aim_list=sorted_list[:self.k] # 取前k个最小元素
       
        count=Counter(i[-1] for i in aim_list) # counter方法，自动计数，返回字典
        return max(count) # 多数表决原则，取数量最多的label

    
#         for i in range(self.k):
#             dis=np.linalg.norm(self.x[i]-x_pre,ord=self.p)
#             knn_list.append((dis,self.y[i]))
            
#         for i in range(self.k,len(self.x)):
#             dis=np.linalg.norm(self.x[i]-x_pre,ord=self.p)
#             max_index=knn_list.index(max(knn_list,key=lambda x:x[0]))
#             if knn_list[max_index][0]>dis:
#                 knn_list[max_index]=(dis,self.y[i])
                
#         knn = [k[-1] for k in knn_list]
#         count_pairs = Counter(knn)
#         max_count = sorted(count_pairs.items(), key=lambda x: x[1])[-1][0]
#         print(max_count)
#         return max_count
      
    def accuracy(self,x_test,y_test): # 计算模型在测试集上的准确率
        count_acc=0
        cnt=0
        for i in x_test:
            if(self.predict(i)==y_test[cnt]):
                cnt+=1
                count_acc+=1
        return count_acc/len(x_test)

In [6]:
model=KNN(X_train,y_train,p=2)

In [7]:
model.accuracy(X_test,y_test)

1.0

In [8]:
X_test[0]

array([6.3, 3.3])

In [11]:
model.predict(X_test[0])

1.0

In [10]:
y_test

array([1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 1.])

# 使用sklearn库实现算法

### 主要参数  
- **n_neighbors**:即为k值
- **weights**:即为权重，默认为uniform不管远近权重都一样，distance则是权重与距离成反比，还可以自定义函数  
- **algorithm**:构建模型的方法， brute-暴力法，kd_tree-kd树实现,ball_tree-球树，auto-自动选择  
- **p**:距离度量，1为曼哈顿距离，2为欧氏距离

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
model2=KNeighborsClassifier()
model2.fit(X_train,y_train) # 使用fit方法开始训练

KNeighborsClassifier()

In [14]:
model2.score(X_test,y_test)

1.0

# kd树的实现

In [20]:
# kd-tree每个结点中主要包含的数据结构如下
class KdNode(object):
    def __init__(self, dom_elt, split, left, right):
        self.dom_elt = dom_elt  # k维向量节点(k维空间中的一个样本点)
        self.split = split  # 整数（进行分割维度的序号）
        self.left = left  # 该结点分割超平面左子空间构成的kd-tree
        self.right = right  # 该结点分割超平面右子空间构成的kd-tree


class KdTree(object):
    def __init__(self, data):
        k = len(data[0])  # 数据维度

        def CreateNode(split, data_set):  # 按第split维划分数据集exset创建KdNode
            if not data_set:  # 数据集为空
                return None
            # key参数的值为一个函数，此函数只有一个参数且返回一个值用来进行比较
            # operator模块提供的itemgetter函数用于获取对象的哪些维的数据，参数为需要获取的数据在对象中的序号
            #data_set.sort(key=itemgetter(split)) # 按要进行分割的那一维数据排序
            data_set.sort(key=lambda x: x[split])
            split_pos = len(data_set) // 2  # //为Python中的整数除法
            median = data_set[split_pos]  # 中位数分割点
            split_next = (split + 1) % k  # cycle coordinates

            # 递归的创建kd树
            return KdNode(
                median,
                split,
                CreateNode(split_next, data_set[:split_pos]),  # 创建左子树
                CreateNode(split_next, data_set[split_pos + 1:]))  # 创建右子树

        self.root = CreateNode(0, data)  # 从第0维分量开始构建kd树,返回根节点


# KDTree的前序遍历
def preorder(root):
    print(root.dom_elt)
    if root.left:  # 节点不为空
        preorder(root.left)
    if root.right:
        preorder(root.right)

In [21]:
data = [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]
kd = KdTree(data)
preorder(kd.root)

[7, 2]
[5, 4]
[2, 3]
[4, 7]
[9, 6]
[8, 1]
